# Lab: Use Spark SQL for ETL

This lab extends the work done in the Spark DataFrames lab, and so assumes you have the data in the relevant places and
have completed those exercises.

--> 1. Create a view from the webpages dataframe you created in the previous lab.

In [0]:
webRDD = sc.textFile("dbfs:/FileStore/tables/extracts/webpage/*"). \
         map( lambda line : line.split()).  \
         map( lambda values :[int(values[0]),values[1],values[2]])

webDF = webRDD.toDF()
webDF = webDF.withColumnRenamed("_1", "id").withColumnRenamed("_2", "webpage").withColumnRenamed("_3", "assoc_files")

webDF.show(5)

+---+--------------------+--------------------+
 id| webpage| assoc_files|
+---+--------------------+--------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|
 2|titanic_2100_sale...|theme.css,code.js...|
 3|meetoo_3.0_sales....|theme.css,code.js...|
 4|meetoo_3.1_sales....|theme.css,code.js...|
 5| ifruit_1_sales.html|theme.css,code.js...|
+---+--------------------+--------------------+
only showing top 5 rows

In [0]:
#webDF = spark.read.parquet("/FileStore/webpage_files/", format="parquet")
webDF.show(5)

+---+--------------------+--------------------+
 id| webpage| assoc_files|
+---+--------------------+--------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|
 2|titanic_2100_sale...|theme.css,code.js...|
 3|meetoo_3.0_sales....|theme.css,code.js...|
 4|meetoo_3.1_sales....|theme.css,code.js...|
 5| ifruit_1_sales.html|theme.css,code.js...|
+---+--------------------+--------------------+
only showing top 5 rows

In [0]:
webDF.createOrReplaceTempView("webview")

In [0]:
# to view this in spark catalog

spark.catalog.listTables()

Out[4]: [Table(name='customers_csv', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='dummy', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='iris_data', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='webview', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

--> 2. We can create a new view from an existing view using SQL's \CREATE OR REPLACE TEMP VIEW <view name> AS"
which can be followed by a SELECT statement correspond to the items to be extracted from the existing view. For
example, to create a new view from the webpages view which contains all the rows corresponding to webpage starting
with s, we could run

In [0]:
spark.sql("CREATE OR REPLACE TEMP VIEW s_webview AS SELECT * FROM webview WHERE webpage LIKE 's%'")
spark.sql("SELECT * FROM s_webview").show(5)

+---+--------------------+--------------------+
 id| webpage| assoc_files|
+---+--------------------+--------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|
 11|sorrento_f21l_sal...|theme.css,code.js...|
 13|sorrento_f23l_sal...|theme.css,code.js...|
 22|sorrento_f24l_sal...|theme.css,code.js...|
 23|sorrento_f20l_sal...|theme.css,code.js...|
+---+--------------------+--------------------+
only showing top 5 rows

In [0]:
spark.catalog.listTables()

Out[6]: [Table(name='customers_csv', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='dummy', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='iris_data', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='s_webview', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='webview', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

--> 3. Check that this view appears to contain the expected items by converting it to a dataframe and showing the contents.

In [0]:
webDF_2 = spark.sql("SELECT * FROM s_webview")
webDF_2.show(5)

+---+--------------------+--------------------+
 id| webpage| assoc_files|
+---+--------------------+--------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|
 11|sorrento_f21l_sal...|theme.css,code.js...|
 13|sorrento_f23l_sal...|theme.css,code.js...|
 22|sorrento_f24l_sal...|theme.css,code.js...|
 23|sorrento_f20l_sal...|theme.css,code.js...|
+---+--------------------+--------------------+
only showing top 5 rows

--> 4. Create a new view which contains the webpage and associated files columns from the view you created in
Point 1.

In [0]:
spark.sql("CREATE OR REPLACE TEMP VIEW webfiles_tmp AS SELECT webpage, assoc_files FROM webview")

spark.sql("SELECT * FROM webfiles_tmp").show(5)

+--------------------+--------------------+
 webpage| assoc_files|
+--------------------+--------------------+
sorrento_f00l_sal...|theme.css,code.js...|
titanic_2100_sale...|theme.css,code.js...|
meetoo_3.0_sales....|theme.css,code.js...|
meetoo_3.1_sales....|theme.css,code.js...|
 ifruit_1_sales.html|theme.css,code.js...|
+--------------------+--------------------+
only showing top 5 rows

--> 5. Check the view was created using listTables.

In [0]:
spark.catalog.listTables()

Out[9]: [Table(name='customers_csv', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='dummy', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='iris_data', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='s_webview', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='webfiles_tmp', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='webview', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

--> 6. This time, we'll manipulate the data using SQL rather than converting to RDDs. We're aiming to use the SQL function
explode which takes an array and separates out its elements into multiple rows (like flatMap). So first we need an
array to be made out of associated files. There's also a function split in SQL, which can be used to split a string
into an array with a specific delimiter. For example, check the contents of the view created by

In [0]:
spark.sql("CREATE OR REPLACE TEMP VIEW spliteg AS SELECT SPLIT(webpage,'_') as split_wp, assoc_files FROM webfiles_tmp")

Out[27]: DataFrame[]

In [0]:
spark.sql("SELECT * FROM spliteg").show(5)

+--------------------+--------------------+
 split_wp| assoc_files|
+--------------------+--------------------+
[sorrento, f00l, ...|theme.css,code.js...|
[titanic, 2100, s...|theme.css,code.js...|
[meetoo, 3.0, sal...|theme.css,code.js...|
[meetoo, 3.1, sal...|theme.css,code.js...|
[ifruit, 1, sales...|theme.css,code.js...|
+--------------------+--------------------+
only showing top 5 rows

In [0]:
spark.sql("CREATE OR REPLACE TEMP VIEW explodeeg AS SELECT EXPLODE(SPLIT(webpage,'_')) as explode_wp, assoc_files FROM webfiles_tmp")
spark.sql("SELECT * FROM explodeeg").show(5)

+----------+--------------------+
explode_wp| assoc_files|
+----------+--------------------+
 sorrento|theme.css,code.js...|
 f00l|theme.css,code.js...|
sales.html|theme.css,code.js...|
 titanic|theme.css,code.js...|
 2100|theme.css,code.js...|
+----------+--------------------+
only showing top 5 rows

--> 7. Create a view which converts the associated files column in the view from Point 4 into an array, splitting on the
commas.

In [0]:
spark.sql("CREATE OR REPLACE TEMP VIEW assoc_split AS SELECT webpage, SPLIT(assoc_files, ',') as split_assfiles FROM webfiles_tmp")
spark.sql("SELECT * FROM assoc_split").show(5)

+--------------------+--------------------+
 webpage| split_assfiles|
+--------------------+--------------------+
sorrento_f00l_sal...|[theme.css, code....|
titanic_2100_sale...|[theme.css, code....|
meetoo_3.0_sales....|[theme.css, code....|
meetoo_3.1_sales....|[theme.css, code....|
 ifruit_1_sales.html|[theme.css, code....|
+--------------------+--------------------+
only showing top 5 rows

In [0]:
# A bit more advanced --> Explode takes only arrays/lists.. so split() will convert the string into an array
spark.sql("CREATE OR REPLACE TEMP VIEW assoc_explode AS SELECT webpage, EXPLODE(SPLIT(assoc_files, ',')) as explode_assfiles FROM webfiles_tmp")
spark.sql("SELECT * FROM assoc_explode").show(5)

+--------------------+-----------------+
 webpage| explode_assfiles|
+--------------------+-----------------+
sorrento_f00l_sal...| theme.css|
sorrento_f00l_sal...| code.js|
sorrento_f00l_sal...|sorrento_f00l.jpg|
titanic_2100_sale...| theme.css|
titanic_2100_sale...| code.js|
+--------------------+-----------------+
only showing top 5 rows

--> 8. Now we move onto using explode. The definition of SQL explode is:

explode(expr) - Separates the elements of array expr into multiple rows, or the elements of map expr into
multiple rows and columns. Unless specified otherwise, uses the default column name col for elements of the
array or key and value for the elements of the map.

Take a look at the new view created by

In [0]:
from pyspark.sql.functions import explode

spliteg_DF = spark.sql("SELECT * FROM spliteg")

spliteg_DF.withColumn("indiv_webfiles", explode("split_wp")).show(5)

+--------------------+--------------------+--------------+
 split_wp| assoc_files|indiv_webfiles|
+--------------------+--------------------+--------------+
[sorrento, f00l, ...|theme.css,code.js...| sorrento|
[sorrento, f00l, ...|theme.css,code.js...| f00l|
[sorrento, f00l, ...|theme.css,code.js...| sales.html|
[titanic, 2100, s...|theme.css,code.js...| titanic|
[titanic, 2100, s...|theme.css,code.js...| 2100|
+--------------------+--------------------+--------------+
only showing top 5 rows

In [0]:
# One more
spark.sql("SELECT * FROM spliteg").withColumn("indiv_webfiles", explode("split_wp")).show(5)

+--------------------+--------------------+--------------+
 split_wp| assoc_files|indiv_webfiles|
+--------------------+--------------------+--------------+
[sorrento, f00l, ...|theme.css,code.js...| sorrento|
[sorrento, f00l, ...|theme.css,code.js...| f00l|
[sorrento, f00l, ...|theme.css,code.js...| sales.html|
[titanic, 2100, s...|theme.css,code.js...| titanic|
[titanic, 2100, s...|theme.css,code.js...| 2100|
+--------------------+--------------------+--------------+
only showing top 5 rows

--> 9. Create a new view from the view you created in Point 7 which explodes the associated files array as was wanted.

In [0]:
spark.sql("SELECT * FROM assoc_split").withColumn("Indiv_files", explode("split_assfiles")).show(5)

+--------------------+--------------------+-----------------+
 webpage| split_assfiles| Indiv_files|
+--------------------+--------------------+-----------------+
sorrento_f00l_sal...|[theme.css, code....| theme.css|
sorrento_f00l_sal...|[theme.css, code....| code.js|
sorrento_f00l_sal...|[theme.css, code....|sorrento_f00l.jpg|
titanic_2100_sale...|[theme.css, code....| theme.css|
titanic_2100_sale...|[theme.css, code....| code.js|
+--------------------+--------------------+-----------------+
only showing top 5 rows

In [0]:
# If we use explode_outer --> It would use null values in the column also, and create a new row with null.
# If we use posexplode    --> It would bring 2 new columns - one is position/index and other is column

In [0]:
# To make it more reader friendly, use drop()
from pyspark.sql.functions import split

spark.sql("SELECT * FROM webfiles_tmp") \
  .withColumn("Indiv_files", explode(split("assoc_files", ","))) \
  .withColumn("Indiv_webs", explode(split("webpage", "_"))) \
  .drop("webpage", "assoc_files") \
  .show(30)
    

+-----------------+----------+
 Indiv_files|Indiv_webs|
+-----------------+----------+
 theme.css| sorrento|
 theme.css| f00l|
 theme.css|sales.html|
 code.js| sorrento|
 code.js| f00l|
 code.js|sales.html|
sorrento_f00l.jpg| sorrento|
sorrento_f00l.jpg| f00l|
sorrento_f00l.jpg|sales.html|
 theme.css| titanic|
 theme.css| 2100|
 theme.css|sales.html|
 code.js| titanic|
 code.js| 2100|
 code.js|sales.html|
 titanic_2100.jpg| titanic|
 titanic_2100.jpg| 2100|
 titanic_2100.jpg|sales.html|
 theme.css| meetoo|
 theme.css| 3.0|
 theme.css|sales.html|
 code.js| meetoo|
 code.js| 3.0|
 code.js|sales.html|
 meetoo_3.0.jpg| meetoo|
 meetoo_3.0.jpg| 3.0|
 meetoo_3.0.jpg|sales.html|
 theme.css| meetoo|
 theme.css| 3.1|
 theme.css|sales.html|
+-----------------+----------+
only showing top 30 rows

## Challenge: perform the ETL solely in DataFrames

The split and explode functions also exist in DataFrames. The optional challenge section will guide you through implementing
the solution purely in DataFrames.

--> 10. Again, we can use a split function to turn the contents of a DataFrame column into an array. Check the contents of

In [0]:
from pyspark.sql.functions import split

splitDF = webDF.withColumn('splitWebpage', split(webDF["webpage"], "_"))
splitDF.show(10)

+---+--------------------+--------------------+--------------------+
 id| webpage| assoc_files| splitWebpage|
+---+--------------------+--------------------+--------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|[sorrento, f00l, ...|
 2|titanic_2100_sale...|theme.css,code.js...|[titanic, 2100, s...|
 3|meetoo_3.0_sales....|theme.css,code.js...|[meetoo, 3.0, sal...|
 4|meetoo_3.1_sales....|theme.css,code.js...|[meetoo, 3.1, sal...|
 5| ifruit_1_sales.html|theme.css,code.js...|[ifruit, 1, sales...|
 6| ifruit_3_sales.html|theme.css,code.js...|[ifruit, 3, sales...|
 7| ifruit_2_sales.html|theme.css,code.js...|[ifruit, 2, sales...|
 8| ifruit_5_sales.html|theme.css,code.js...|[ifruit, 5, sales...|
 9|titanic_1000_sale...|theme.css,code.js...|[titanic, 1000, s...|
 10|meetoo_1.0_sales....|theme.css,code.js...|[meetoo, 1.0, sal...|
+---+--------------------+--------------------+--------------------+
only showing top 10 rows

--> 11. Create DF with an additional column which corresponds to an array made of the comma separated elements of the
original associated files column of the webpages DataFrame.

In [0]:
from pyspark.sql.functions import split

splitDF_ass = webDF.withColumn('splitAssFiles', split(webDF["assoc_files"], ","))
splitDF_ass.show(10)

+---+--------------------+--------------------+--------------------+
 id| webpage| assoc_files| splitAssFiles|
+---+--------------------+--------------------+--------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|[theme.css, code....|
 2|titanic_2100_sale...|theme.css,code.js...|[theme.css, code....|
 3|meetoo_3.0_sales....|theme.css,code.js...|[theme.css, code....|
 4|meetoo_3.1_sales....|theme.css,code.js...|[theme.css, code....|
 5| ifruit_1_sales.html|theme.css,code.js...|[theme.css, code....|
 6| ifruit_3_sales.html|theme.css,code.js...|[theme.css, code....|
 7| ifruit_2_sales.html|theme.css,code.js...|[theme.css, code....|
 8| ifruit_5_sales.html|theme.css,code.js...|[theme.css, code....|
 9|titanic_1000_sale...|theme.css,code.js...|[theme.css, code....|
 10|meetoo_1.0_sales....|theme.css,code.js...|[theme.css, code....|
+---+--------------------+--------------------+--------------------+
only showing top 10 rows

--> 12. Now consider the contents of the following DataFrame:

In [0]:
from pyspark.sql.functions import explode
explodedDF = splitDF.withColumn('explodedWebpage', explode(splitDF.splitWebpage))
explodedDF.show(10)

+---+--------------------+--------------------+--------------------+---------------+
 id| webpage| assoc_files| splitWebpage|explodedWebpage|
+---+--------------------+--------------------+--------------------+---------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|[sorrento, f00l, ...| sorrento|
 1|sorrento_f00l_sal...|theme.css,code.js...|[sorrento, f00l, ...| f00l|
 1|sorrento_f00l_sal...|theme.css,code.js...|[sorrento, f00l, ...| sales.html|
 2|titanic_2100_sale...|theme.css,code.js...|[titanic, 2100, s...| titanic|
 2|titanic_2100_sale...|theme.css,code.js...|[titanic, 2100, s...| 2100|
 2|titanic_2100_sale...|theme.css,code.js...|[titanic, 2100, s...| sales.html|
 3|meetoo_3.0_sales....|theme.css,code.js...|[meetoo, 3.0, sal...| meetoo|
 3|meetoo_3.0_sales....|theme.css,code.js...|[meetoo, 3.0, sal...| 3.0|
 3|meetoo_3.0_sales....|theme.css,code.js...|[meetoo, 3.0, sal...| sales.html|
 4|meetoo_3.1_sales....|theme.css,code.js...|[meetoo, 3.1, sal...| meetoo|
+---+--------------------+--------------------+--------------------+---------------+
only showing top 10 rows

--> 13. Enhance your DataFrame from Point 11 with the exploded array column.

In [0]:
splitDF_ass.withColumn("Exploded_Ass_Files", explode("splitAssFiles")).show(10)

+---+--------------------+--------------------+--------------------+------------------+
 id| webpage| assoc_files| splitAssFiles|Exploded_Ass_Files|
+---+--------------------+--------------------+--------------------+------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|[theme.css, code....| theme.css|
 1|sorrento_f00l_sal...|theme.css,code.js...|[theme.css, code....| code.js|
 1|sorrento_f00l_sal...|theme.css,code.js...|[theme.css, code....| sorrento_f00l.jpg|
 2|titanic_2100_sale...|theme.css,code.js...|[theme.css, code....| theme.css|
 2|titanic_2100_sale...|theme.css,code.js...|[theme.css, code....| code.js|
 2|titanic_2100_sale...|theme.css,code.js...|[theme.css, code....| titanic_2100.jpg|
 3|meetoo_3.0_sales....|theme.css,code.js...|[theme.css, code....| theme.css|
 3|meetoo_3.0_sales....|theme.css,code.js...|[theme.css, code....| code.js|
 3|meetoo_3.0_sales....|theme.css,code.js...|[theme.css, code....| meetoo_3.0.jpg|
 4|meetoo_3.1_sales....|theme.css,code.js...|[theme.css, code....| theme.css|
+---+--------------------+--------------------+--------------------+------------------+
only showing top 10 rows